## 2. Athena, the analytics AWS service

### 2.1 Create two S3 buckets : One to store the average temperature per country data, and an empty one for the results of the queries.

In [ ]:
import boto3
import uuid
import os
import pandas as pd
from dotenv import load_dotenv

Make sure to create a .env file with your credentials:

```bash
SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
ACCESS_KEY_ID=XXXXXXXXXXXX
```

In [ ]:
# CSV_DATA = "data/GlobalLandTemperaturesByCity.csv"
CSV_DATA = "data/GlobalTemperatures.csv"
CSV_RESULTS = "data/results.csv"
# name in the bucket
FILE_NAME_KEY = "upload_file.csv"

load_dotenv()

SECRET_ACCESS_KEY = os.getenv('SECRET_ACCESS_KEY')
ACCESS_KEY_ID = os.getenv('ACCESS_KEY_ID')
REGION_NAME = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')

session = boto3.Session(
    aws_secret_access_key=SECRET_ACCESS_KEY,
    aws_access_key_id=ACCESS_KEY_ID,
    region_name=REGION_NAME
)

s3_resource = session.resource('s3')
s3_client = session.client('s3')

In [ ]:
def create_bucket_name(bucket_prefix):
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_resource):
    bucket_name = create_bucket_name(bucket_prefix)
    bucket_response = s3_resource.create_bucket(Bucket=bucket_name)
    return bucket_name, bucket_response

def get_buckets(s3_client):
    response = s3_client.list_buckets()
    bucket_data = ""
    bucket_4_results = ""
    for bucket in response['Buckets']:
        if "bucket-4-results" in bucket["Name"]:
            bucket_4_results = bucket["Name"]
        elif "bucket-data" in bucket["Name"]:
            bucket_data = bucket["Name"]
    return bucket_data, bucket_4_results


In [ ]:
# Check if buckets already exist
bucket_data, bucket_4_results = get_buckets(s3_client)

if bucket_4_results == "" :
    bucket_4_results, ig=create_bucket("bucket-4-results",s3_resource)
if bucket_data == "":
    bucket_data, ig=create_bucket("bucket-data",s3_resource)
    push_file(bucket_data, CSV_DATA, FILE_NAME_KEY,s3_resource)

print("bucket date: ", bucket_data)
print("bucket 4 res: ", bucket_4_results)

### 2.2 Get familiar with the following by reading the Boto3 Documentation on Athena:

- **Named queries**: creation, deletion, batching, listing.
- **Prepared queries**: creation, deletion, batching, listing and update.
- **Query execution**: Start, Stop, Get...


### Named queries

Resource that specifies an Amazon Athena saved query, where QueryString contains the SQL query statements that make up the query.

#### Creation

Creates a named query in the specified workgroup. Requires that you have access to the workgroup.

```python
response = client.create_named_query(
    Name='string',
    Description='string',
    Database='string',
    QueryString='string',
    ClientRequestToken='string',
    WorkGroup='string'
)
```

Returns a dictionary with the `NamedQueryId`

```json
{
    'NamedQueryId': 'string'
}
```

#### Deletion

Deletes the named query if you have access to the workgroup in which the query was saved.

```python
response = client.delete_named_query(
    NamedQueryId='string'
)
```

If the `NamedQuery` is deleted correctly it returns an empty dictionary. 

#### Batching

Returns the details of a single named query or a list of up to 50 queries, which you provide as an array of query ID strings.

```python
response = client.batch_get_named_query(
    NamedQueryIds=[
        'string',
    ]
)
```

Returns a dictionary with this structure
```json
{
    'NamedQueries': [
        {
            'Name': 'string',
            'Description': 'string',
            'Database': 'string',
            'QueryString': 'string',
            'NamedQueryId': 'string',
            'WorkGroup': 'string'
        },
    ],
    'UnprocessedNamedQueryIds': [
        {
            'NamedQueryId': 'string',
            'ErrorCode': 'string',
            'ErrorMessage': 'string'
        },
    ]
}
```

#### Listing

Provides a list of available query IDs only for queries saved in the specified workgroup. Requires that you have access to the specified workgroup.

```python
response = client.list_named_queries(
    NextToken='string',
    MaxResults=123,
    WorkGroup='string'
)
```

Returns a dictionary with this structure
```json
{
    'NamedQueryIds': [
        'string',
    ],
    'NextToken': 'string'
}
```

### Prepared queries (ParameterizedQueries / PreparedStatement)

Parameterized queries to re-run the same query with different parameter values at execution time and help prevent SQL injection attacks. In Athena, parameterized queries can take the form of execution parameters in any DML query or SQL prepared statements.

#### Creation

Creates a prepared statement for use with SQL queries in Athena.

```python
response = client.create_prepared_statement(
    StatementName='string',
    WorkGroup='string',
    QueryStatement='string',
    Description='string'
)
```

#### Deletion

Deletes the prepared statement with the specified name from the specified workgroup.

```python
response = client.delete_prepared_statement(
    StatementName='string',
    WorkGroup='string'
)
```

#### Batching

Returns the details of a single prepared statement or a list of up to 256 prepared statements for the array of prepared statement names that you provide.

```python
response = client.batch_get_prepared_statement(
    PreparedStatementNames=[
        'string',
    ],
    WorkGroup='string'
)
```

Returns a dictionary with this structure.

```json
{
    'PreparedStatements': [
        {
            'StatementName': 'string',
            'QueryStatement': 'string',
            'WorkGroupName': 'string',
            'Description': 'string',
            'LastModifiedTime': datetime(2015, 1, 1)
        },
    ],
    'UnprocessedPreparedStatementNames': [
        {
            'StatementName': 'string',
            'ErrorCode': 'string',
            'ErrorMessage': 'string'
        },
    ]
}
```

#### Listing 

Lists the prepared statements in the specified workgroup.

```python
response = client.list_prepared_statements(
    WorkGroup='string',
    NextToken='string',
    MaxResults=123
)
```

Returns a dictionary with this structure.

```json
{
    'PreparedStatements': [
        {
            'StatementName': 'string',
            'LastModifiedTime': datetime(2015, 1, 1)
        },
    ],
    'NextToken': 'string'
}
```

#### Update

Updates a prepared statement.

```python
response = client.update_prepared_statement(
    StatementName='string',
    WorkGroup='string',
    QueryStatement='string',
    Description='string'
)
```


### Query execution

Information about a single instance of a query execution.

#### Start

Runs the SQL query statements contained in the `Query`. Requires you to have access to the workgroup in which the query ran.

```python
response = client.start_query_execution(
    QueryString='string',
    ClientRequestToken='string',
    QueryExecutionContext={
        'Database': 'string',
        'Catalog': 'string'
    },
    ResultConfiguration={
        'OutputLocation': 'string',
        'EncryptionConfiguration': {
            'EncryptionOption': 'SSE_S3'|'SSE_KMS'|'CSE_KMS',
            'KmsKey': 'string'
        },
        'ExpectedBucketOwner': 'string',
        'AclConfiguration': {
            'S3AclOption': 'BUCKET_OWNER_FULL_CONTROL'
        }
    },
    WorkGroup='string',
    ExecutionParameters=[
        'string',
    ],
    ResultReuseConfiguration={
        'ResultReuseByAgeConfiguration': {
            'Enabled': True|False,
            'MaxAgeInMinutes': 123
        }
    }
)
```

Returns a dictionary with these structure

```json
{
    'QueryExecutionId': 'string'
}
```

#### Stop

Stops a query execution. Requires you to have access to the workgroup in which the query ran.

```python
response = client.stop_query_execution(
    QueryExecutionId='string'
)
```

#### Get

Returns information about a single execution of a query if you have access to the workgroup in which the query ran.

```python
response = client.get_query_execution(
    QueryExecutionId='string'
)
```

Returns a dict with these attributes
```json
{
    'QueryExecution': {
        'QueryExecutionId': 'string',
        'Query': 'string',
        'StatementType': 'DDL'|'DML'|'UTILITY',
        'ResultConfiguration': {
            'OutputLocation': 'string',
            'EncryptionConfiguration': {
                'EncryptionOption': 'SSE_S3'|'SSE_KMS'|'CSE_KMS',
                'KmsKey': 'string'
            },
            'ExpectedBucketOwner': 'string',
            'AclConfiguration': {
                'S3AclOption': 'BUCKET_OWNER_FULL_CONTROL'
            }
        },
        'ResultReuseConfiguration': {
            'ResultReuseByAgeConfiguration': {
                'Enabled': True|False,
                'MaxAgeInMinutes': 123
            }
        },
        'QueryExecutionContext': {
            'Database': 'string',
            'Catalog': 'string'
        },
        'Status': {
            'State': 'QUEUED'|'RUNNING'|'SUCCEEDED'|'FAILED'|'CANCELLED',
            'StateChangeReason': 'string',
            'SubmissionDateTime': datetime(2015, 1, 1),
            'CompletionDateTime': datetime(2015, 1, 1),
            'AthenaError': {
                'ErrorCategory': 123,
                'ErrorType': 123,
                'Retryable': True|False,
                'ErrorMessage': 'string'
            }
        },
        'Statistics': {
            'EngineExecutionTimeInMillis': 123,
            'DataScannedInBytes': 123,
            'DataManifestLocation': 'string',
            'TotalExecutionTimeInMillis': 123,
            'QueryQueueTimeInMillis': 123,
            'QueryPlanningTimeInMillis': 123,
            'ServiceProcessingTimeInMillis': 123,
            'ResultReuseInformation': {
                'ReusedPreviousResult': True|False
            }
        },
        'WorkGroup': 'string',
        'EngineVersion': {
            'SelectedEngineVersion': 'string',
            'EffectiveEngineVersion': 'string'
        },
        'ExecutionParameters': [
            'string',
        ]
    }
}
```


### 2.3 Create three functions that query the data bucket and store the results in the results bucket.

In [ ]:
#
# Execute a query in a S3 bucket with a SQL expression
# e.g: SELECT * from S3Object LIMIT 4
#
def get_from_s3(s3_client, bucket_name, file_name, sql_expression):
    resp = s3_client.select_object_content(
        Bucket = bucket_name,
        Key = file_name,
        Expression = sql_expression,
        ExpressionType = 'SQL',
        InputSerialization = {'CSV': {'FileHeaderInfo': 'Use'}},
        OutputSerialization = {'CSV': {}}
    )
    file1 = open(CSV_RESULTS,"a")
    for event in resp['Payload']:
        if 'Records' in event:
            tmp = event['Records']['Payload'].decode()
            file1.write(tmp)
            print(event['Records']['Payload'].decode())
    
    file1.close()

def push_file(client_bucket_name, file_name, file_name_key, s3_resource):
    s3_resource.Bucket(client_bucket_name).upload_file(Filename = file_name, Key = file_name_key)

In [ ]:
SQL_QUERY = "SELECT * from S3Object LIMIT 4"

get_from_s3(s3_client, bucket_data, FILE_NAME_KEY, SQL_QUERY)

push_file(bucket_4_results, CSV_RESULTS, CSV_RESULTS, s3_resource)